# LRMC

In [ ]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def svt(mat, tau):
    u, s, v = np.linalg.svd(mat, full_matrices = 0)
    vec = s - tau
    vec[vec < 0] = 0
    return u @ np.diag(vec) @ v

def LRMC(dense_mat, sparse_mat, rho, maxiter = 100):
    
    pos_train = np.where(sparse_mat != 0)
    mat_train = sparse_mat[pos_train]
    pos_test = np.where((sparse_mat == 0) & (dense_mat != 0))
    mat_test = dense_mat[pos_test]
    
    X = sparse_mat.copy()
    Z = sparse_mat.copy()
    T = sparse_mat.copy()
    del dense_mat, sparse_mat
    show_iter = 100
    for it in range(maxiter):
        Z = svt(X + T / rho, 1 / rho)
        X = Z - T / rho
        X[pos_train] = mat_train
        T = T - rho * (Z - X)
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(mat_test, X[pos_test]))
            print(compute_rmse(mat_test, X[pos_test]))
            print()
    return X

### Tune hyperparameter

In [ ]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('pems-w{}.npz'.format(t))['arr_0'],
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.9
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)
np.savez_compressed('dense_mat.npz', dense_mat)
np.savez_compressed('sparse_mat.npz', sparse_mat)

import cupy as np

dense_mat = np.load('dense_mat.npz')['arr_0'][:, : 14 * 288]
sparse_mat = np.load('sparse_mat.npz')['arr_0'][:, : 14 * 288]

import time
for lmbda in [1e-4, 1e-5]:
    print('lmbda = {}'.format(lmbda))
    start = time.time()
    alpha = np.ones(3) / 3
    tensor_hat = LRMC(dense_tensor, sparse_tensor, lmbda)
    end = time.time()
    print('Running time: %d seconds.'%(end - start))
    print()

### Report imputation results

In [ ]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('pems-w{}.npz'.format(t))['arr_0'],
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.9
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)
np.savez_compressed('dense_mat.npz', dense_mat)
np.savez_compressed('sparse_mat.npz', sparse_mat)

import cupy as np

dense_mat = np.load('dense_mat.npz')['arr_0'][:, 14 * 288 :]
sparse_mat = np.load('sparse_mat.npz')['arr_0'][:, 14 * 288 :]

import time
for lmbda in [1e-4, 1e-5]:
    print('lmbda = {}'.format(lmbda))
    start = time.time()
    alpha = np.ones(3) / 3
    tensor_hat = LRMC(dense_tensor, sparse_tensor, lmbda)
    end = time.time()
    print('Running time: %d seconds.'%(end - start))
    print()

# HaLRTC

In [ ]:
import numpy as np
from numpy.linalg import inv as inv

def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

def mat2ten(mat, tensor_size, mode):
    index = list()
    index.append(mode)
    for i in range(tensor_size.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, tensor_size[index].tolist(), order = 'F'), 0, mode)

def svt_tnn(mat, alpha, rho, theta):
    """This is a Numpy dependent singular value thresholding (SVT) process."""
    u, s, v = np.linalg.svd(mat, full_matrices = 0)
    vec = s.copy()
    vec[theta :] = s[theta :] - alpha / rho
    vec[vec < 0] = 0
    return u @ np.diag(vec) @ v

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, 
         epsilon = 1e-4, maxiter = 100):
    """Low-Rank Tenor Completion with Truncated Nuclear Norm, LRTC-TNN."""
    
    dim = np.array(sparse_tensor.shape)
    pos_missing = np.where(sparse_tensor == 0)
    pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    
    X = np.zeros((int(len(dim)), int(dim[0]), int(dim[1]), int(dim[2])))
    T = np.zeros((int(len(dim)), int(dim[0]), int(dim[1]), int(dim[2]))) 
    Z = sparse_tensor.copy()
    last_tensor = sparse_tensor.copy()
    snorm = np.sqrt(np.sum(sparse_tensor ** 2))
    it = 0
    while True:
        rho = min(rho * 1.05, 1e5)
        for k in range(len(dim)):
            X[k] = mat2ten(svt_tnn(ten2mat(Z - T[k] / rho, k), alpha[k], rho, theta), dim, k)
        Z[pos_missing] = np.mean(X + T / rho, axis = 0)[pos_missing]
        temp = np.zeros((int(len(dim)), int(dim[0]), int(dim[1]), int(dim[2])))
        for i in range(int(len(dim))):
            temp[i] = Z
        T = T + rho * (X - temp)
        tensor_hat = np.einsum('k, kmnt -> mnt', alpha, X)
        tol = np.sqrt(np.sum((tensor_hat - last_tensor) ** 2)) / snorm
        last_tensor = tensor_hat.copy()
        it += 1
        if (it + 1) % 100 == 0:
            print('Iter: {}'.format(it + 1))
            print('MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], tensor_hat[pos_test])))
            print('RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], tensor_hat[pos_test])))
            print()
        if (tol < epsilon) or (it >= maxiter):
            break

    print('Imputation MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], tensor_hat[pos_test])))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], tensor_hat[pos_test])))
    print()
    
    return tensor_hat

### Tune hyperparameter

In [ ]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('pems-w{}.npz'.format(t))['arr_0'],
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.3
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)
np.savez_compressed('dense_mat.npz', dense_mat)
np.savez_compressed('sparse_mat.npz', sparse_mat)

import cupy as np

dense_mat = np.load('dense_mat.npz')['arr_0'][:, : 14 * 288]
sparse_mat = np.load('sparse_mat.npz')['arr_0'][:, : 14 * 288]
dense_tensor = mat2ten(dense_mat, np.array([dim1, 288, 14]), 0)
sparse_tensor = mat2ten(sparse_mat, np.array([dim1, 288, 14]), 0)
del dense_mat, sparse_mat

import time
for lmbda in [1e-4, 1e-5]:
    for theta in [0]:
        print('lmbda = {}'.format(lmbda))
        print('theta = {}'.format(theta))
        start = time.time()
        alpha = np.ones(3) / 3
        tensor_hat = LRTC(dense_tensor, sparse_tensor,
                          alpha, lmbda, theta)
        end = time.time()
        print('Running time: %d seconds.'%(end - start))
        print()

### Report imputation results

In [ ]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('pems-w{}.npz'.format(t))['arr_0'],
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.3
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)
np.savez_compressed('dense_mat.npz', dense_mat)
np.savez_compressed('sparse_mat.npz', sparse_mat)

import cupy as np

dense_mat = np.load('dense_mat.npz')['arr_0'][:, 14 * 288 :]
sparse_mat = np.load('sparse_mat.npz')['arr_0'][:, 14 * 288 :]
dense_tensor = mat2ten(dense_mat, np.array([dim1, 288, 14]), 0)
sparse_tensor = mat2ten(sparse_mat, np.array([dim1, 288, 14]), 0)
del dense_mat, sparse_mat

import time
for lmbda in [1e-5]:
    for theta in [0]:
        print('lmbda = {}'.format(lmbda))
        print('theta = {}'.format(theta))
        start = time.time()
        alpha = np.ones(3) / 3
        tensor_hat = LRTC(dense_tensor, sparse_tensor,
                          alpha, lmbda, theta)
        end = time.time()
        print('Running time: %d seconds.'%(end - start))
        print()

# LRTC-TNN

In [ ]:
import numpy as np
from numpy.linalg import inv as inv

def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

def mat2ten(mat, tensor_size, mode):
    index = list()
    index.append(mode)
    for i in range(tensor_size.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, tensor_size[index].tolist(), order = 'F'), 0, mode)

def svt_tnn(mat, alpha, rho, theta):
    """This is a Numpy dependent singular value thresholding (SVT) process."""
    u, s, v = np.linalg.svd(mat, full_matrices = 0)
    vec = s.copy()
    vec[theta :] = s[theta :] - alpha / rho
    vec[vec < 0] = 0
    return u @ np.diag(vec) @ v

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, 
         epsilon = 1e-4, maxiter = 100):
    """Low-Rank Tenor Completion with Truncated Nuclear Norm, LRTC-TNN."""
    
    dim = np.array(sparse_tensor.shape)
    pos_missing = np.where(sparse_tensor == 0)
    pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    
    X = np.zeros((int(len(dim)), int(dim[0]), int(dim[1]), int(dim[2])))
    T = np.zeros((int(len(dim)), int(dim[0]), int(dim[1]), int(dim[2]))) 
    Z = sparse_tensor.copy()
    last_tensor = sparse_tensor.copy()
    snorm = np.sqrt(np.sum(sparse_tensor ** 2))
    it = 0
    while True:
        rho = min(rho * 1.05, 1e5)
        for k in range(len(dim)):
            X[k] = mat2ten(svt_tnn(ten2mat(Z - T[k] / rho, k), alpha[k], rho, theta), dim, k)
        Z[pos_missing] = np.mean(X + T / rho, axis = 0)[pos_missing]
        temp = np.zeros((int(len(dim)), int(dim[0]), int(dim[1]), int(dim[2])))
        for i in range(int(len(dim))):
            temp[i] = Z
        T = T + rho * (X - temp)
        tensor_hat = np.einsum('k, kmnt -> mnt', alpha, X)
        tol = np.sqrt(np.sum((tensor_hat - last_tensor) ** 2)) / snorm
        last_tensor = tensor_hat.copy()
        it += 1
        if (it + 1) % 100 == 0:
            print('Iter: {}'.format(it + 1))
            print('MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], tensor_hat[pos_test])))
            print('RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], tensor_hat[pos_test])))
            print()
        if (tol < epsilon) or (it >= maxiter):
            break

    print('Imputation MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], tensor_hat[pos_test])))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], tensor_hat[pos_test])))
    print()
    
    return tensor_hat

### Tune hyperparameter

In [ ]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('pems-w{}.npz'.format(t))['arr_0'],
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.3
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)
np.savez_compressed('dense_mat.npz', dense_mat)
np.savez_compressed('sparse_mat.npz', sparse_mat)

import cupy as np

dense_mat = np.load('dense_mat.npz')['arr_0'][:, : 14 * 288]
sparse_mat = np.load('sparse_mat.npz')['arr_0'][:, : 14 * 288]
dense_tensor = mat2ten(dense_mat, np.array([dim1, 288, 14]), 0)
sparse_tensor = mat2ten(sparse_mat, np.array([dim1, 288, 14]), 0)
del dense_mat, sparse_mat

import time
for lmbda in [1e-4, 1e-5]:
    for theta in [5, 10, 15, 20, 25]:
        print('lmbda = {}'.format(lmbda))
        print('theta = {}'.format(theta))
        start = time.time()
        alpha = np.ones(3) / 3
        tensor_hat = LRTC(dense_tensor, sparse_tensor,
                          alpha, lmbda, theta)
        end = time.time()
        print('Running time: %d seconds.'%(end - start))
        print()

### Report imputation results

In [ ]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('pems-w{}.npz'.format(t))['arr_0'],
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.3
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)
np.savez_compressed('dense_mat.npz', dense_mat)
np.savez_compressed('sparse_mat.npz', sparse_mat)

import cupy as np

dense_mat = np.load('dense_mat.npz')['arr_0'][:, 14 * 288 :]
sparse_mat = np.load('sparse_mat.npz')['arr_0'][:, 14 * 288 :]
dense_tensor = mat2ten(dense_mat, np.array([dim1, 288, 14]), 0)
sparse_tensor = mat2ten(sparse_mat, np.array([dim1, 288, 14]), 0)
del dense_mat, sparse_mat

import time
for lmbda in [1e-5]:
    for theta in [25]:
        print('lmbda = {}'.format(lmbda))
        print('theta = {}'.format(theta))
        start = time.time()
        alpha = np.ones(3) / 3
        tensor_hat = LRTC(dense_tensor, sparse_tensor,
                          alpha, lmbda, theta)
        end = time.time()
        print('Running time: %d seconds.'%(end - start))
        print()

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>